In [2]:
import os

In [3]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction/research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction'

In [6]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tejas05in"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "9efcb5c7b79d0e949378459b922b1462a80fa413"

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml , create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        This function returns the ModelEvaluationConfig object that contains all the configuration parameters required for model evaluation.

        Args:
            None

        Returns:
            ModelEvaluationConfig: The ModelEvaluationConfig object containing all the configuration parameters

        """
        config = self.config.model_evaluation
        params = self.params.GradientBoostedTreesLearner
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
        )

        return model_evaluation_config

In [24]:
import os
import pandas as pd
from urllib.parse import urlparse
import mlflow
import numpy as np
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from ThyroidProject.utils.common import save_json
from mlflow.data.pandas_dataset import PandasDataset

In [25]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    

    def log_into_mlflow(self):
        """
        This function logs the metrics and model into mlflow.

        Args:
            None

        Returns:
            None

        """
        mlflow.tensorflow.autolog()

        # Load the test data
        test_data = pd.read_csv(self.config.test_data_path)

        # Convert the pandas dataframe to a tf.data.Dataset
        test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
            test_data, label=self.config.target_column)

        # Load the model
        model = tf.keras.models.load_model(self.config.model_path)

        # Compile the model
        model.compile(metrics=["accuracy"])

        # Evaluate the model on the test data
        evaluation = model.evaluate(test_ds, return_dict=True)
        
        # Save the metrics as a local file
        # scores = {"accuray": evaluation["accuracy"], "loss": evaluation["loss"]}
        save_json(path=Path(self.config.metric_file_name), data=evaluation)

        # Set the mlflow tracking uri
        mlflow.set_registry_uri(self.config.mlflow_uri)

        # Get the type of the tracking uri
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        dataset: PandasDataset = mlflow.data.from_pandas(
            test_data)
        
        # Start a mlflow run
        with mlflow.start_run():
            
            #log data
            mlflow.log_input(dataset, context="testing")

            # Log the parameters
            mlflow.log_params(self.config.all_params)

            # Log the metrics
            mlflow.log_metric("loss", evaluation["loss"])
            mlflow.log_metric("accuray", evaluation["accuracy"])

            # Register the model
            # There are other ways to use the model registry , which depends on the use case
            # Please refer to the documentation for more information
            # https://mlflow.org/docs/latest/model-registry.html #api-workflow
            if tracking_uri_type_store != 'file':
                mlflow.tensorflow.log_model(
                    model, 'model', registered_model_name="GradientBoostedTreesModel", conda_env="conda.yaml")
            else:
                mlflow.tensorflow.log_model(model, 'model')

In [26]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-27 21:01:29,164:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-01-27 21:01:29,165:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-27 21:01:29,166:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-01-27 21:01:29,167:INFO:common:created directory at :artifacts]
[2024-01-27 21:01:29,167:INFO:common:created directory at :artifacts/model_evaluation]
[2024-01-27 21:01:29,172:WARNING:tf_logging:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.]


[INFO 24-01-27 21:01:29.4168 IST kernel.cc:1233] Loading model from path artifacts/model_trainer/model/assets/ with prefix 30989f3147be440e
[INFO 24-01-27 21:01:29.4215 IST decision_forest.cc:660] Model loaded with 111 root(s), 4117 node(s), and 14 input feature(s).
[INFO 24-01-27 21:01:29.4215 IST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-01-27 21:01:29.4215 IST kernel.cc:1061] Use fast generic engine


2/2 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.9865


/home/tejas/MLProj/Thyroid-Disease-Prediction/env/lib/python3.11/site-packages/mlflow/data/digest_utils.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]


[2024-01-27 21:03:29,796:DEBUG:retry:Incremented Retry for (url='/tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/create'): Retry(total=4, connect=5, read=4, redirect=5, status=5)]
[2024-01-27 21:03:29,799:WARNING:connectionpool:Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/create]
[2024-01-27 21:03:29,801:DEBUG:connectionpool:Starting new HTTPS connection (4): dagshub.com:443]
[2024-01-27 21:03:30,440:DEBUG:connectionpool:https://dagshub.com:443 "POST /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/create HTTP/1.1" 200 None]
[2024-01-27 21:03:30,443:INFO:common:json file saved at:artifacts/model_evaluation/metrics.json]


...

[2024-01-27 21:03:42,438:INFO:builder_impl:Assets written to: /tmp/tmpmv74r39p/model/data/model/assets]
[2024-01-27 21:03:42,929:DEBUG:connectionpool:https://dagshub.com:443 "GET /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/get?run_uuid=0f0305a6fb454424a88bd4a456476cfd&run_id=0f0305a6fb454424a88bd4a456476cfd HTTP/1.1" 200 None]
[2024-01-27 21:03:43,543:DEBUG:connectionpool:https://dagshub.com:443 "PUT /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/dbd409d79d1546fdbfc410acf1df7960/0f0305a6fb454424a88bd4a456476cfd/artifacts/model/MLmodel HTTP/1.1" 200 None]
[2024-01-27 21:03:44,041:DEBUG:connectionpool:https://dagshub.com:443 "PUT /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/dbd409d79d1546fdbfc410acf1df7960/0f0305a6fb454424a88bd4a456476cfd/artifacts/model/conda.yaml HTTP/1.1" 200 None]
[2024-01-27 21:03:44,568:DEBUG:connectionpool:https://dagshub.com:443 "PUT /tejas05in/Thyroid-Disease-Prediction.mlfl

Successfully registered model 'GradientBoostedTreesModel'.


[2024-01-27 21:03:58,699:DEBUG:connectionpool:https://dagshub.com:443 "GET /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/get?run_uuid=0f0305a6fb454424a88bd4a456476cfd&run_id=0f0305a6fb454424a88bd4a456476cfd HTTP/1.1" 200 None]
[2024-01-27 21:03:59,109:DEBUG:connectionpool:https://dagshub.com:443 "POST /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/model-versions/create HTTP/1.1" 200 None]


2024/01/27 21:03:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GradientBoostedTreesModel, version 1
Created version '1' of model 'GradientBoostedTreesModel'.


[2024-01-27 21:03:59,620:DEBUG:connectionpool:https://dagshub.com:443 "POST /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/update HTTP/1.1" 200 None]
